# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner and combine it with our other data skills. In this lab you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer follow-up questions. 


### Outline:

1. Determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create Python functions to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results.

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- What are the 5 businesses with the highest average ratings?
- What are the 5 businesses with the lowest average ratings?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- How many businesses have a rating above 4.5?
- How many businesses have a rating below 3?
- Return the text of the oldes review in the table.
- Return the overall rating of the business with the oldest review. 
- Find the highest rated business and return text of the newest review of the three.
- Find the lowest rated business and return text of the newest review of the three.  


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [ ]:
## Connect to DB server on AWS

In [ ]:
## Create new DB 

In [ ]:
# Create a table for the Businesses

In [ ]:
# Create a table for the reviews

## Part 2: Create ETL pipeline for the business data from the API

In [ ]:
# write a function to make a call to the API

In [ ]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

In [1]:
# Write a function to take your parsed data and insert it into the DB

## Part 3: Create ETL pipeline for the review data from the API

In [ ]:
# write a query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that take a business id 
# and makes a call to the API for reivews
# then parse out the relevant information

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
## Part 4: Write SQL queries that will answer the questions posed. 

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter.***

In [31]:
# function to get yelp business reviews
# BEFORE refactoring
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
#     data = response.json()['businesses']
#     return data
    return response.json()

def all_results(url_params, api_key):
    num = yelp_call(url_params, config.key)['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.extend(yelp_call(url_params, api_key)['businesses'])
        # could also insert into DB now, instead
        # of waiting
        time.sleep(.1) #Wait a second
        cur += 50
    return results

In [21]:
term = 'burgers'
location = 'Manhattan'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }


In [22]:
import config
import requests

In [24]:
# This does all the work to get the restuarant reviews:
results = all_results(url_params, config.key)

8900 total matches found.


In [333]:
business_ids = []
for i in results:
    business_ids.append(i['id'])

In [15]:
business_names = []
for i in results:
    business_names.append(i['name'])
business_names

['Black Iron Burger',
 'Black Tap Craft Burgers & Beer',
 'Burger & Lobster',
 "Benson's NYC",
 'Burger & Lobster',
 'The Burgary',
 'Bareburger',
 'Holy Cow',
 'BH Kitchen & Bar',
 'Judge Roy Bean Public House',
 'Burger Joint',
 'JG Melon',
 "Paul's Da Burger Joint",
 'Breakroom',
 'Emily - West Village',
 "Whitman's",
 'Bareburger',
 '5 Napkin Burger',
 'Burger One',
 'Snax Spot',
 'Brindle Room',
 'Hero Certified Burgers',
 'Black Burger',
 'Harvest Kitchen',
 "Flip'n Toss",
 "Ruby's Cafe",
 'Burgers By Honest Chops',
 'Royale',
 'Burger Inc Nyc',
 'Geteburgers',
 'Shake Shack',
 'Black Iron Burger',
 'Copper John’s Pub + Kitchen',
 "Richie's Burger Joint",
 'Superiority Burger',
 "Juanchi's Burger",
 "Bill's Bar & Burger",
 'Whitmans Hudson Yards',
 'Black Burger',
 'The Flying Cock',
 'Bareburger',
 "Ruby's - Murray Hill",
 'Tanner Smiths',
 'Burger & Barrel',
 "P.J. Clarke's",
 "Gebhard's Beer Culture",
 "Junior's Restaurant & Bakery - 45th St.",
 'The Usual',
 'YN',
 'Hendriks'

In [334]:
business_ratings = []
for i in results:
    business_ratings.append(i['rating'])

In [335]:
business_prices = []
for i in results:
    if 'price' in i.keys():
        business_prices.append(i['price'])
    else:
        business_prices.append(None)

In [142]:
def get_bus_recs(results):
    bus_recs_list = []
    for rec in results:
        id = rec.get('id')
        name = rec.get('name', None)
        rating = rec.get('rating', None)
        price = rec.get('price', None)
        bus_recs_list.append(dict(id=id, name=name, rating=rating, price=price))
        
    return bus_recs_list


In [42]:
import config

In [44]:
import importlib
importlib.reload(config)

<module 'config' from '/Users/markbrennan/bootcamp/nyc-mhtn-ds-071519-lectures/week-2/config.py'>

In [52]:
# Business Insert BEFORE Refactoring

from datetime import date, datetime, timedelta
import mysql.connector
from mysql.connector import errorcode

DB_NAME = 'Yelp'

cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = DB_NAME
)
cursor = cnx.cursor()

#tomorrow = datetime.now().date() + timedelta(days=1)


add_business = ("INSERT INTO businesses "
               "(id, name, rating, price) "
               "VALUES (%s, %s, %s, %s)")

values = (business_ids[0], business_names[0], business_ratings[0], business_prices[0])

cursor.execute(add_business, values)

# Make sure data is committed to the database
cnx.commit()

cursor.close()
cnx.close()



In [202]:
# Populate DB with restaurant records
# AFTER Refactoring
def populate_db(records, table_name, config_params):
    INSERT_STRING = ("INSERT INTO {table_name} "
               "(id, name, rating, price) "
               "VALUES (%s, %s, %s, %s)")
    INSERT_STR = f'INSERT INTO {table_name} (id, name, rating, price) ' 
    VALUE_STR = 'VALUES (%s, %s, %s, %s)'
    INSERT_QUERY = INSERT_STR + VALUE_STR
    print(INSERT_STR)
    print(VALUE_STR)
    print(INSERT_QUERY)
    conn = yelp.get_db_conn(config_params)
    if not conn:
        print('No DB connection!')
        return
    try:
        cursor = conn.cursor()
        for rec in records:
            values = tuple(rec.values())
            cursor.execute(INSERT_QUERY, values)

        conn.commit()
        
    finally:
        conn.close()    

In [177]:
## CHECKING FOR DUPES!
import collections
dupes = [item for item, count in collections.Counter(business_ids).items() if count > 1]

In [178]:
dupes

['bpjDo7svAFD10eszaOPQHw']

In [179]:
business_ids.index('bpjDo7svAFD10eszaOPQHw')

99

In [180]:
dupe_slots = []
for i in range(len(business_ids)):
    if business_ids[i] == 'bpjDo7svAFD10eszaOPQHw':
        dupe_slots.append(i)

In [181]:
dupe_slots

[99, 100]

In [ ]:
for i in dupes:
    for j in range(len(business_ids)):
        if business_ids[j] == i:
            del results[j]

In [185]:
import copy
clean_results = copy.deepcopy(results)

In [54]:
# Populate DB with restaurant records
# BEFORE Refactoring

def insert_into_db():
    cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = "Yelp")
    cursor = cnx.cursor()
    for i in range(1,1000):
        add_business = ("INSERT INTO businesses "
               "(id, name, rating, price) "
               "VALUES (%s, %s, %s, %s)")
        values = (business_ids[i], business_names[i], business_ratings[i], business_prices[i])
        cursor.execute(add_business, values)
    # Make sure data is committed to the database
    cnx.commit()

    cursor.close()
    cnx.close()
    

In [55]:
insert_into_db()

In [59]:
def yelp_review_call(api_key, bus_id):
# GET https://api.yelp.com/v3/businesses/{id}/reviews

    url = f'https://api.yelp.com/v3/businesses/{bus_id}/reviews'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    
    response = requests.get(url, headers=headers)
    
#     data = response.json()['businesses']
#     return data
    return response.json()


In [63]:
# code to get Yelp reviews
# BEFORE refactoring
reviews_all_dict = dict.fromkeys(business_ids,0)

for i in business_ids:
    reviews_all_dict[i] = yelp_review_call(config.key, i)
len(reviews_all_dict)

999

In [65]:
# Reviews API took a while to run, so save to disk!
import json
with open('reviews.json', 'w') as fp:
    json.dump(reviews_all_dict, fp)


In [223]:
# Create our review records
# Before refactoring
review_list = []
for key in reviews_data.keys():
    bus_id = key
    reviews = reviews_data[key]['reviews']
    for item in reviews:
        review_list.append(dict(id=item['id'], 
                             text=item['text'], 
                             rating=item['rating'], 
                             creation_dt=item['time_created'],
                             bus_id=bus_id))

In [226]:
# PROCESS OUR REVIEWS LIST HERE:

In [233]:
import config
import yelp

In [257]:
db_conn = yelp.get_db_conn(config)

In [258]:
# Code to insert review data into DB
# BEFPRE refactoring
INSERT_STR = f'INSERT INTO reviews (id, text, rating, creation_date, bus_id) '
VALUE_STR = 'VALUES (%s, %s, %s, %s, %s)'
INSERT_QUERY = INSERT_STR + VALUE_STR

#    conn = get_db_conn(config_params)
if not db_conn:
    print('No DB connection!')
try:
    cursor = db_conn.cursor()
    for review in review_list:
        values = tuple(review.values())
        try:
            cursor.execute(INSERT_QUERY, values)
        except: # assuming bad data, but want to continue
            print ('row: ', values)
            continue


finally:
    db_conn.commit()
    db_conn.close()


row:  ('GKyBfU-XnyMCwq6yE3qDeQ', "Oh! My! God! Chef David! \nOne of the Best experiences and meals I have had in nyc/Brooklyn.  It's a neighborhood gem. The restaurant itself is cozy and...", 5, '2019-01-20 18:24:22', 'nKjEA5nW5zm2MjD2ukdTtQ')
row:  ('b4loAJG3uneonwjDN150rQ', "The pork\nThe lasagna\nThe salad\nThe donkey sculpture\nThe hazelnut dessert\nYou son of a bitch, David you've done it again.", 5, '2019-01-20 18:06:49', 'nKjEA5nW5zm2MjD2ukdTtQ')
row:  ('brIH9WqEBRepq8HnZk3mig', 'Visiting from California we happened to stubble across this restaurant and what an experience. Everyone was on on deck from staff to make we were kept feed...', 5, '2019-07-15 09:24:24', 'nKjEA5nW5zm2MjD2ukdTtQ')
row:  ('G6d_UEodWPmf9mARjH57mw', 'Short Stories is everything I want in a restaurant. They have vegan/vegetarian options but also food with meat to keep everyone happy. The dishes are unique...', 5, '2019-05-05 06:23:54', 'QsIAkIjzoAOtfgrABBe2ow')
row:  ('5l3g2CGpF6kvfBwCePS3dQ', 'Came here for

In [125]:
import importlib
importlib.reload(config)

<module 'config' from '/Users/markbrennan/bootcamp/nyc-mhtn-ds-071519-lectures/week-2/config.py'>

In [159]:
importlib.reload(yelp)

<module 'yelp' from '/Users/markbrennan/bootcamp/nyc-mhtn-ds-071519-lectures/week-2/yelp.py'>

In [161]:
records = yelp.get_bus_recs(results)

In [162]:
len(records)

1000

In [ ]:
def yelp_review_call(api_key, bus_id):
    url = f'https://api.yelp.com/v3/businesses/{bus_id}/reviews'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    
    response = requests.get(url, headers=headers)
    
    return response.json()

In [ ]:
reviews_all_dict = dict.fromkeys([rec['id'] for rec in records],0)

for bus_id in reviews_all_dict.keys():
    reviews_all_dict[bus_id] = yelp_review_call(config.key, bus_id)

In [158]:
importlib.reload(config)

<module 'config' from '/Users/markbrennan/bootcamp/nyc-mhtn-ds-071519-lectures/week-2/config.py'>

In [ ]:
yelp.get

In [128]:
conn.close()

In [139]:
reviews_all_dict['fgtw4swOHXEFwZpBCG05rQ']

{'reviews': [{'id': 'FGHH7p9ZL40ogeMl0vPJbA',
   'url': 'https://www.yelp.com/biz/black-iron-burger-new-york-8?adjust_creative=iA-hJdCOqADgjC47eVjdcA&hrid=FGHH7p9ZL40ogeMl0vPJbA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iA-hJdCOqADgjC47eVjdcA',
   'text': 'The burger is good and the slogan is the worst slogan ever. It makes me think of the old joke What is the difference between meat and fish? If you beat your...',
   'rating': 4,
   'time_created': '2019-07-21 19:33:39',
   'user': {'id': 'wCrg8GPUGMqP6fczjuM4_w',
    'profile_url': 'https://www.yelp.com/user_details?userid=wCrg8GPUGMqP6fczjuM4_w',
    'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/mq0hgPFn0b81RI3cUuKiQQ/o.jpg',
    'name': 'Tim C.'}},
  {'id': '64MLXs6q4PsvcFz5PA67GA',
   'url': 'https://www.yelp.com/biz/black-iron-burger-new-york-8?adjust_creative=iA-hJdCOqADgjC47eVjdcA&hrid=64MLXs6q4PsvcFz5PA67GA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iA-hJdCOqADgjC

In [135]:
reviews_all_dict['fgtw4swOHXEFwZpBCG05rQ']['reviews'][0]

{'id': 'FGHH7p9ZL40ogeMl0vPJbA',
 'url': 'https://www.yelp.com/biz/black-iron-burger-new-york-8?adjust_creative=iA-hJdCOqADgjC47eVjdcA&hrid=FGHH7p9ZL40ogeMl0vPJbA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iA-hJdCOqADgjC47eVjdcA',
 'text': 'The burger is good and the slogan is the worst slogan ever. It makes me think of the old joke What is the difference between meat and fish? If you beat your...',
 'rating': 4,
 'time_created': '2019-07-21 19:33:39',
 'user': {'id': 'wCrg8GPUGMqP6fczjuM4_w',
  'profile_url': 'https://www.yelp.com/user_details?userid=wCrg8GPUGMqP6fczjuM4_w',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/mq0hgPFn0b81RI3cUuKiQQ/o.jpg',
  'name': 'Tim C.'}}

In [136]:
reviews_all_dict['fgtw4swOHXEFwZpBCG05rQ']['reviews'][0]['id']

'FGHH7p9ZL40ogeMl0vPJbA'

In [137]:
reviews_all_dict['fgtw4swOHXEFwZpBCG05rQ']['reviews'][0]['text']

'The burger is good and the slogan is the worst slogan ever. It makes me think of the old joke What is the difference between meat and fish? If you beat your...'

In [138]:
reviews_all_dict['fgtw4swOHXEFwZpBCG05rQ']['reviews'][0]['time_created']

'2019-07-21 19:33:39'

In [284]:
import importlib
importlib.reload(yelp)

<module 'yelp' from '/Users/markbrennan/bootcamp/nyc-mhtn-ds-071519-lectures/week-2/yelp.py'>

In [285]:
help(yelp.yelp_review_call)

Help on function yelp_review_call in module yelp:

yelp_review_call(api_key, bus_id)
    yelp_review_call(api_key, bus_id):
    Get the Yelp reviews for the business id
    Params:
        api_key: Yelp api key (from config)
        bus_id: the business id for which to get reviews
    Returns:
        Json dict of reviews



In [286]:
test_reviews = yelp.get_reviews(records, test_lim=5)

TypeError: 'int' object is not subscriptable

In [290]:
test_reviews['fgtw4swOHXEFwZpBCG05rQ']['reviews']

[{'id': 'FGHH7p9ZL40ogeMl0vPJbA',
  'url': 'https://www.yelp.com/biz/black-iron-burger-new-york-8?adjust_creative=iA-hJdCOqADgjC47eVjdcA&hrid=FGHH7p9ZL40ogeMl0vPJbA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iA-hJdCOqADgjC47eVjdcA',
  'text': 'The burger is good and the slogan is the worst slogan ever. It makes me think of the old joke What is the difference between meat and fish? If you beat your...',
  'rating': 4,
  'time_created': '2019-07-21 19:33:39',
  'user': {'id': 'wCrg8GPUGMqP6fczjuM4_w',
   'profile_url': 'https://www.yelp.com/user_details?userid=wCrg8GPUGMqP6fczjuM4_w',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/mq0hgPFn0b81RI3cUuKiQQ/o.jpg',
   'name': 'Tim C.'}},
 {'id': '64MLXs6q4PsvcFz5PA67GA',
  'url': 'https://www.yelp.com/biz/black-iron-burger-new-york-8?adjust_creative=iA-hJdCOqADgjC47eVjdcA&hrid=64MLXs6q4PsvcFz5PA67GA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iA-hJdCOqADgjC47eVjdcA',
  'text': '

In [323]:
importlib.reload(yelp)

<module 'yelp' from '/Users/markbrennan/bootcamp/nyc-mhtn-ds-071519-lectures/week-2/yelp.py'>

In [324]:
test_dict = yelp.get_reviews(records=small_recs, test_lim=5)

In [332]:
for i in test_dict.keys():
    print(test_dict[i].get('reviews'))

[{'id': 'FGHH7p9ZL40ogeMl0vPJbA', 'url': 'https://www.yelp.com/biz/black-iron-burger-new-york-8?adjust_creative=iA-hJdCOqADgjC47eVjdcA&hrid=FGHH7p9ZL40ogeMl0vPJbA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iA-hJdCOqADgjC47eVjdcA', 'text': 'The burger is good and the slogan is the worst slogan ever. It makes me think of the old joke What is the difference between meat and fish? If you beat your...', 'rating': 4, 'time_created': '2019-07-21 19:33:39', 'user': {'id': 'wCrg8GPUGMqP6fczjuM4_w', 'profile_url': 'https://www.yelp.com/user_details?userid=wCrg8GPUGMqP6fczjuM4_w', 'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/mq0hgPFn0b81RI3cUuKiQQ/o.jpg', 'name': 'Tim C.'}}, {'id': '64MLXs6q4PsvcFz5PA67GA', 'url': 'https://www.yelp.com/biz/black-iron-burger-new-york-8?adjust_creative=iA-hJdCOqADgjC47eVjdcA&hrid=64MLXs6q4PsvcFz5PA67GA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iA-hJdCOqADgjC47eVjdcA', 'text': '100% recommend this plac

AttributeError: 'int' object has no attribute 'get'

In [291]:
def get_reviews(records, test_lim=None):
    """get_reviews(records):
    Get our Yelp reviews from the restuarant records
    Params:
        Records: Restaurant records containing business IDs
    Returns:
        reviews: list of review dicts
    """

    limit = True if test_lim else False
    lim_count = test_lim if test_lim else 0
    iter_count = 0

    reviews_all_dict = dict.fromkeys([rec['id'] for rec in records], 0)

    for bus_id in reviews_all_dict.keys():
        if not limit:
            reviews_all_dict[bus_id] = yelp_review_call(config.key, bus_id)
        else:
            if iter_count <= lim_count:
                reviews_all_dict[bus_id] = yelp_review_call(config.key, bus_id)
                iter_count += 1

    review_list = []
    for key in reviews_all_dict.keys():
        bus_id = key
        reviews = reviews_all_dict[key]['reviews']
        for item in reviews:
            review_list.append(dict(id=item['id'],
                text=item['text'],
                rating=item['rating'],
                creation_dt=item['time_created'],
                bus_id=bus_id))

    return review_list


In [309]:
reviews_all_dict = dict.fromkeys([rec['id'] for rec in small_recs], 0)

In [312]:
reviews_all_dict['fgtw4swOHXEFwZpBCG05rQ'] = yelp_review_call(config.key, 'fgtw4swOHXEFwZpBCG05rQ')

In [320]:
for key in reviews_all_dict.keys():
    print(key)
    print(reviews_all_dict[key])

print(reviews_all_dict['fgtw4swOHXEFwZpBCG05rQ']['reviews'])

fgtw4swOHXEFwZpBCG05rQ
{'reviews': [{'id': 'FGHH7p9ZL40ogeMl0vPJbA', 'url': 'https://www.yelp.com/biz/black-iron-burger-new-york-8?adjust_creative=iA-hJdCOqADgjC47eVjdcA&hrid=FGHH7p9ZL40ogeMl0vPJbA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iA-hJdCOqADgjC47eVjdcA', 'text': 'The burger is good and the slogan is the worst slogan ever. It makes me think of the old joke What is the difference between meat and fish? If you beat your...', 'rating': 4, 'time_created': '2019-07-21 19:33:39', 'user': {'id': 'wCrg8GPUGMqP6fczjuM4_w', 'profile_url': 'https://www.yelp.com/user_details?userid=wCrg8GPUGMqP6fczjuM4_w', 'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/mq0hgPFn0b81RI3cUuKiQQ/o.jpg', 'name': 'Tim C.'}}, {'id': '64MLXs6q4PsvcFz5PA67GA', 'url': 'https://www.yelp.com/biz/black-iron-burger-new-york-8?adjust_creative=iA-hJdCOqADgjC47eVjdcA&hrid=64MLXs6q4PsvcFz5PA67GA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iA-hJdCOqADgjC47eVjdcA'

In [314]:
reviews_all_dict['fgtw4swOHXEFwZpBCG05rQ']['reviews']

[{'id': 'FGHH7p9ZL40ogeMl0vPJbA',
  'url': 'https://www.yelp.com/biz/black-iron-burger-new-york-8?adjust_creative=iA-hJdCOqADgjC47eVjdcA&hrid=FGHH7p9ZL40ogeMl0vPJbA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iA-hJdCOqADgjC47eVjdcA',
  'text': 'The burger is good and the slogan is the worst slogan ever. It makes me think of the old joke What is the difference between meat and fish? If you beat your...',
  'rating': 4,
  'time_created': '2019-07-21 19:33:39',
  'user': {'id': 'wCrg8GPUGMqP6fczjuM4_w',
   'profile_url': 'https://www.yelp.com/user_details?userid=wCrg8GPUGMqP6fczjuM4_w',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/mq0hgPFn0b81RI3cUuKiQQ/o.jpg',
   'name': 'Tim C.'}},
 {'id': '64MLXs6q4PsvcFz5PA67GA',
  'url': 'https://www.yelp.com/biz/black-iron-burger-new-york-8?adjust_creative=iA-hJdCOqADgjC47eVjdcA&hrid=64MLXs6q4PsvcFz5PA67GA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=iA-hJdCOqADgjC47eVjdcA',
  'text': '

In [293]:
len(records)

1000

In [305]:
small_recs = records[0:50]

In [306]:
len(small_recs)

50

In [297]:
small_recs[0]

{'id': 'fgtw4swOHXEFwZpBCG05rQ',
 'name': 'Black Iron Burger',
 'rating': 4.0,
 'price': '$$'}

In [308]:
small_reviews = yelp.get_reviews(small_recs)

In [307]:
tuple(small_reviews[0].values())

('FGHH7p9ZL40ogeMl0vPJbA',
 'The burger is good and the slogan is the worst slogan ever. It makes me think of the old joke What is the difference between meat and fish? If you beat your...',
 4,
 '2019-07-21 19:33:39',
 'fgtw4swOHXEFwZpBCG05rQ')

In [304]:
yelp.populate_reviews(small_reviews, config_params=config, table_name='test_reviews')

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```